In [19]:
import pandas as pd
import requests
import time
import os
from datetime import datetime

index_frame = pd.read_csv('index_master.csv')
filtered_df = index_frame[index_frame['year'] >= 2017]

# Find player-year combinations with exactly one team
unique_teams = (
    filtered_df.groupby(['nba_id', 'year'])['team_id']
    .nunique()
    .reset_index()
    .rename(columns={'team_id': 'team_count'})
)
single_team_players = unique_teams[unique_teams['team_count'] == 1]
result_df = pd.merge(filtered_df, single_team_players[['nba_id', 'year']], on=['nba_id', 'year'])
player_team_map = result_df[['nba_id', 'team_id', 'year']].drop_duplicates()
player_team_map = player_team_map.rename(columns={'nba_id': 'player_id', 'year': 'season_end_year'})
def format_date_to_url(date):
    # Convert date from YYYYMMDD to datetime object
    date_obj = datetime.strptime(str(date), '%Y%m%d')
    
    # Format the date as MM%2FDD%2FYYYY
    formatted_date = date_obj.strftime('%m%%2F%d%%2F%Y')
    
    return formatted_date
def pull_data(url):
    headers = {
        "Host": "stats.nba.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://stats.nba.com/"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    json = response.json()

    if len(json["resultSets"]) == 1:
        data = json["resultSets"][0]["rowSet"]
        columns = json["resultSets"][0]["headers"]
    else:
        data = json["resultSets"]["rowSet"]
        columns = json["resultSets"]["headers"][1]['columnNames']
    
    return pd.DataFrame.from_records(data, columns=columns)

def get_matchups_for_date(season, date, mode='Offense', is_playoffs=False):
    fdate = format_date_to_url(date)
    stype = 'Playoffs' if is_playoffs else 'Regular%20Season'
    url = (
        'https://stats.nba.com/stats/leagueseasonmatchups?'
        f'DateFrom={fdate}&DateTo={fdate}&DefPlayerID=&DefTeamID=&LeagueID=00'
        f'&Matchup={mode}&OffPlayerID=&OffTeamID=&Outcome=&PORound=0&PerMode=Totals'
        f'&Season={season}&SeasonType={stype}'
    )
    return pull_data(url)

def create_game_team_mappings(matchups_df, date):
    """
    Create team-to-opponent mappings for a specific game date.
    Since teams play against each other on the same date, we can map team pairs.
    """
    # Get all known team assignments for this date
    known_teams = set()
    
    if 'OFF_TEAM_ID' in matchups_df.columns:
        known_teams.update(matchups_df['OFF_TEAM_ID'].dropna().unique())
    if 'DEF_TEAM_ID' in matchups_df.columns:
        known_teams.update(matchups_df['DEF_TEAM_ID'].dropna().unique())
    
    # Create opponent mapping by finding team pairs that appear together
    team_opponent_map = {}
    
    # Method 1: Use rows where both offensive and defensive teams are known
    if 'OFF_TEAM_ID' in matchups_df.columns and 'DEF_TEAM_ID' in matchups_df.columns:
        both_known = matchups_df[(matchups_df['OFF_TEAM_ID'].notna()) & 
                                (matchups_df['DEF_TEAM_ID'].notna())].copy()
        
        if not both_known.empty:
            for _, row in both_known.iterrows():
                off_team = row['OFF_TEAM_ID']
                def_team = row['DEF_TEAM_ID']
                
                # Map each team to its opponent
                team_opponent_map[off_team] = def_team
                team_opponent_map[def_team] = off_team
    
    # Method 2: If we don't have direct mappings, infer from team co-occurrence patterns
    if len(team_opponent_map) == 0:
        # Group by potential game identifiers and find team pairs
        if 'SEASON_ID' in matchups_df.columns or other_grouping_available:
            # This would require more sophisticated logic based on available columns
            # For now, we'll rely on the direct mapping method above
            pass
    
    print(f"   Created {len(team_opponent_map)//2} team pair mappings for {date}")
    return team_opponent_map

def map_teams_to_matchups(matchups_df, player_team_map, season_end_year, date, mode='Defense'):
    """
    Enhanced team mapping using game-level team pairing logic
    """
    matchups_df = matchups_df.copy()
    matchups_df['season_end_year'] = season_end_year
    
    # Step 1: Initial mapping from player_team_map
    off_map = player_team_map.rename(columns={'player_id': 'OFF_PLAYER_ID', 'team_id': 'OFF_TEAM_ID'})
    def_map = player_team_map.rename(columns={'player_id': 'DEF_PLAYER_ID', 'team_id': 'DEF_TEAM_ID'})
    
    matchups_df = pd.merge(matchups_df, off_map, on=['OFF_PLAYER_ID', 'season_end_year'], how='left')
    matchups_df = pd.merge(matchups_df, def_map, on=['DEF_PLAYER_ID', 'season_end_year'], how='left')
    
    initial_off_coverage = matchups_df['OFF_TEAM_ID'].notna().mean()
    initial_def_coverage = matchups_df['DEF_TEAM_ID'].notna().mean()
    
    print(f"   Initial mapping - OFF_TEAM_ID: {initial_off_coverage:.1%}, DEF_TEAM_ID: {initial_def_coverage:.1%}")
    
    # Step 2: Create game-level team opponent mappings
    team_opponent_map = create_game_team_mappings(matchups_df, date)
    
    # Step 3: Fill missing team IDs using opponent mappings
    if team_opponent_map:
        # Fill missing offensive team IDs using known defensive team IDs
        mask_missing_off = matchups_df['OFF_TEAM_ID'].isna() & matchups_df['DEF_TEAM_ID'].notna()
        if mask_missing_off.any():
            matchups_df.loc[mask_missing_off, 'OFF_TEAM_ID'] = matchups_df.loc[mask_missing_off, 'DEF_TEAM_ID'].map(team_opponent_map)
            filled_off = matchups_df.loc[mask_missing_off, 'OFF_TEAM_ID'].notna().sum()
            print(f"   Filled {filled_off} missing OFF_TEAM_ID using opponent mapping")
        
        # Fill missing defensive team IDs using known offensive team IDs
        mask_missing_def = matchups_df['DEF_TEAM_ID'].isna() & matchups_df['OFF_TEAM_ID'].notna()
        if mask_missing_def.any():
            matchups_df.loc[mask_missing_def, 'DEF_TEAM_ID'] = matchups_df.loc[mask_missing_def, 'OFF_TEAM_ID'].map(team_opponent_map)
            filled_def = matchups_df.loc[mask_missing_def, 'DEF_TEAM_ID'].notna().sum()
            print(f"   Filled {filled_def} missing DEF_TEAM_ID using opponent mapping")
    
    # Step 4: Propagate consistent team assignments within the same date
    # All matchups for the same player on the same date should have the same team
    
    # Propagate offensive team consistency
    off_player_teams = (matchups_df[matchups_df['OFF_TEAM_ID'].notna()]
                       .groupby('OFF_PLAYER_ID')['OFF_TEAM_ID']
                       .first()
                       .to_dict())
    
    mask_propagate_off = matchups_df['OFF_TEAM_ID'].isna()
    matchups_df.loc[mask_propagate_off, 'OFF_TEAM_ID'] = (
        matchups_df.loc[mask_propagate_off, 'OFF_PLAYER_ID'].map(off_player_teams)
    )
    
    # Propagate defensive team consistency
    def_player_teams = (matchups_df[matchups_df['DEF_TEAM_ID'].notna()]
                       .groupby('DEF_PLAYER_ID')['DEF_TEAM_ID']
                       .first()
                       .to_dict())
    
    mask_propagate_def = matchups_df['DEF_TEAM_ID'].isna()
    matchups_df.loc[mask_propagate_def, 'DEF_TEAM_ID'] = (
        matchups_df.loc[mask_propagate_def, 'DEF_PLAYER_ID'].map(def_player_teams)
    )
    
    # Step 5: Second round of opponent mapping after propagation
    if team_opponent_map:
        # Update team_opponent_map with any new teams we've mapped
        updated_team_opponent_map = create_game_team_mappings(matchups_df, date)
        if len(updated_team_opponent_map) > len(team_opponent_map):
            team_opponent_map.update(updated_team_opponent_map)
            print(f"   Updated opponent mapping with {len(updated_team_opponent_map)//2} team pairs")
        
        # Second round of filling
        mask_missing_off = matchups_df['OFF_TEAM_ID'].isna() & matchups_df['DEF_TEAM_ID'].notna()
        if mask_missing_off.any():
            matchups_df.loc[mask_missing_off, 'OFF_TEAM_ID'] = matchups_df.loc[mask_missing_off, 'DEF_TEAM_ID'].map(team_opponent_map)
        
        mask_missing_def = matchups_df['DEF_TEAM_ID'].isna() & matchups_df['OFF_TEAM_ID'].notna()
        if mask_missing_def.any():
            matchups_df.loc[mask_missing_def, 'DEF_TEAM_ID'] = matchups_df.loc[mask_missing_def, 'OFF_TEAM_ID'].map(team_opponent_map)
    
    # Final coverage report
    final_off_coverage = matchups_df['OFF_TEAM_ID'].notna().mean()
    final_def_coverage = matchups_df['DEF_TEAM_ID'].notna().mean()
    
    print(f"   Final mapping - OFF_TEAM_ID: {final_off_coverage:.1%} (+{final_off_coverage-initial_off_coverage:.1%})")
    print(f"   Final mapping - DEF_TEAM_ID: {final_def_coverage:.1%} (+{final_def_coverage-initial_def_coverage:.1%})")
    
    # Report unmapped players
    unmapped_off = matchups_df[matchups_df['OFF_TEAM_ID'].isna()]['OFF_PLAYER_ID'].unique()
    unmapped_def = matchups_df[matchups_df['DEF_TEAM_ID'].isna()]['DEF_PLAYER_ID'].unique()
    
    if len(unmapped_off) > 0:
        print(f"   Still unmapped offensive players: {len(unmapped_off)}")
    if len(unmapped_def) > 0:
        print(f"   Still unmapped defensive players: {len(unmapped_def)}")
    
    return matchups_df

def enhance_existing_file_mappings(filename, player_team_map):
    """
    Apply enhanced team mapping to existing files to improve coverage
    """
    if not os.path.exists(filename):
        return
        
    print(f"🔧 Enhancing existing file: {filename}")
    df = pd.read_csv(filename)
    
    # Group by date and apply enhanced mapping
    enhanced_dfs = []
    for date in df['game_date'].unique():
        date_df = df[df['game_date'] == date].copy()
        season_end_year = date_df['season_end_year'].iloc[0] if 'season_end_year' in date_df.columns else None
        
        if season_end_year:
            enhanced_df = map_teams_to_matchups(date_df, player_team_map, season_end_year, date)
            enhanced_dfs.append(enhanced_df)
        else:
            enhanced_dfs.append(date_df)
    
    if enhanced_dfs:
        enhanced_final = pd.concat(enhanced_dfs, ignore_index=True)
        enhanced_final.to_csv(filename, index=False)
        print(f"   ✅ Enhanced file saved")

# === Pipeline Start ===
df = pd.read_csv("game_dates.csv")
df['season_end_year'] = df['season'].str.split('-').str[0].astype(int) + 1
filtered_df = df[df['season_end_year'] >= 2023]
unique_dates = filtered_df[['date', 'season', 'season_end_year', 'playoffs']].drop_duplicates()
unique_dates = unique_dates.head(1)

# Create game_id mapping lookup from game_dates file
game_dates_lookup = df[['date', 'TEAM_ID', 'GAME_ID']].drop_duplicates()
print(f"📋 Loaded {len(game_dates_lookup)} game date-team-game_id mappings")

mode = 'Offense'  # Change to 'Defense' if needed

# Optional: create output folder
output_dir = 'matchup_outputs'
os.makedirs(output_dir, exist_ok=True)

# Track global team mappings across all dates for pattern learning
global_team_mappings = {}

grouped = unique_dates.groupby('season_end_year')
for year, group in grouped:
    filename = os.path.join(output_dir, f"matchups_{mode.lower()}_{year}.csv")

    # Step 1: Check for existing file and enhance it first
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename)
        scraped_dates = set(existing_df['game_date'].unique())
        print(f"📁 Found existing file for {year} with {len(scraped_dates)} dates already scraped.")
        
        # Enhance existing file with improved team mapping
        enhance_existing_file_mappings(filename, player_team_map)
        
        # Reload the enhanced file
        existing_df = pd.read_csv(filename)
        
        # Learn team patterns from existing data
        if 'OFF_TEAM_ID' in existing_df.columns and 'DEF_TEAM_ID' in existing_df.columns:
            for date in existing_df['game_date'].unique():
                date_df = existing_df[existing_df['game_date'] == date]
                date_mappings = create_game_team_mappings(date_df, date)
                global_team_mappings.update(date_mappings)
        
        print(f"   Learned {len(global_team_mappings)//2} team pair patterns from existing data")
    else:
        scraped_dates = set()

    # Step 2: Filter group to exclude scraped dates
    group_to_scrape = group[~group['date'].isin(scraped_dates)]

    if group_to_scrape.empty:
        print(f"⏭️  All dates already scraped for {year}. Skipping.")
        continue

    all_matchups = []

    for _, row in group_to_scrape.iterrows():
        date = row['date']
        season = row['season']
        is_playoffs = row['playoffs']

        print(f"📅 Pulling {mode} matchup for {date} ({'Playoffs' if is_playoffs else 'Regular Season'})")

        try:
            matchups_df = get_matchups_for_date(season, date, mode=mode, is_playoffs=is_playoffs)
            
            # Enhanced team mapping with game-level logic
            matchups_df = map_teams_to_matchups(matchups_df, player_team_map, year, date, mode)
            
            # Add metadata
            matchups_df['date'] = date
            matchups_df['season'] = season
            matchups_df['mode'] = mode
            matchups_df['playoffs'] = is_playoffs
            
            # Learn from this date's team mappings for future use
            date_mappings = create_game_team_mappings(matchups_df, date)
            global_team_mappings.update(date_mappings)
            matchups_df=matchups_df.merge(game_dates_lookup)
            all_matchups.append(matchups_df)
            
        except Exception as e:
            print(f"❌ Error on {date} ({season}) - {e}")

        time.sleep(0.3)  # Rate limiting

    if all_matchups:
        new_df = pd.concat(all_matchups, ignore_index=True)
        new_df.drop_duplicates(inplace=True)

        # Step 3: Append to existing file if it exists
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)  # Reload enhanced file
            final_df = pd.concat([existing_df, new_df], ignore_index=True).drop_duplicates()
        else:
            final_df = new_df

        final_df.to_csv(filename, index=False)
        print(f"✅ Saved/updated file for {year}: {filename}")
        
        # Report final team mapping coverage
        if 'OFF_TEAM_ID' in final_df.columns:
            off_coverage = final_df['OFF_TEAM_ID'].notna().mean()
            print(f"   Final offensive team mapping coverage: {off_coverage:.1%}")
        if 'DEF_TEAM_ID' in final_df.columns:
            def_coverage = final_df['DEF_TEAM_ID'].notna().mean()
            print(f"   Final defensive team mapping coverage: {def_coverage:.1%}")
            
    else:
        print(f"⚠️ No new data collected for {year}.")

print(f"\n✅ Done. Enhanced team mapping completed with {len(global_team_mappings)//2} total team pair patterns learned.")
print("📊 Team mapping improvements:")
print("   - Game-level opponent mapping: Map missing team IDs using known opponent teams")
print("   - Player consistency: Ensure same player has same team for all matchups on same date")
print("   - Cross-validation: Use both offensive and defensive team data to fill gaps")
print("   - Pattern learning: Build global team pairing database across all processed dates")


📋 Loaded 73279 game date-team-game_id mappings
📅 Pulling Offense matchup for 20221018 (Regular Season)
   Initial mapping - OFF_TEAM_ID: 86.5%, DEF_TEAM_ID: 86.5%
   Created 15 team pair mappings for 20221018
   Filled 15825 missing OFF_TEAM_ID using opponent mapping
   Filled 15805 missing DEF_TEAM_ID using opponent mapping
   Created 15 team pair mappings for 20221018
   Final mapping - OFF_TEAM_ID: 100.0% (+13.5%)
   Final mapping - DEF_TEAM_ID: 100.0% (+13.5%)
   Created 15 team pair mappings for 20221018
✅ Saved/updated file for 2023: matchup_outputs/matchups_offense_2023.csv
   Final offensive team mapping coverage: 100.0%
   Final defensive team mapping coverage: 100.0%

✅ Done. Enhanced team mapping completed with 15 total team pair patterns learned.
📊 Team mapping improvements:
   - Game-level opponent mapping: Map missing team IDs using known opponent teams
   - Player consistency: Ensure same player has same team for all matchups on same date
   - Cross-validation: Use both 

In [10]:
df.columns

Index(['GAME_ID', 'TEAM_ID', 'HTM', 'VTM', 'date', 'season', 'playoffs',
       'team', 'opp_team', 'season_end_year'],
      dtype='object')

In [22]:
import pandas as pd
import requests
import time

def pull_data(url):
    headers = {
        "Host": "stats.nba.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://stats.nba.com/"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    json = response.json()

    if len(json["resultSets"]) == 1:
        data = json["resultSets"][0]["rowSet"]
        columns = json["resultSets"][0]["headers"]
    else:
        data = json["resultSets"]["rowSet"]
        columns = json["resultSets"]["headers"][1]['columnNames']
    
    df = pd.DataFrame.from_records(data, columns=columns)
    return df

def get_omatchups_by_date(season, date, mode='Offense', ps=False):
    """
    Pulls offensive or defensive matchup data for a specific date in a given season.
    mode: 'Offense' or 'Defense'
    """
    stype = 'Playoffs' if ps else 'Regular%20Season'
    url = (
        'https://stats.nba.com/stats/leagueseasonmatchups?'
        f'DateFrom={date}&DateTo={date}&DefPlayerID=&DefTeamID=&LeagueID=00'
        f'&Matchup={mode}&OffPlayerID=&OffTeamID=&Outcome=&PORound=0&PerMode=Totals'
        f'&Season={season}&SeasonType={stype}'
    )
    return pull_data(url)

# === Pipeline ===

# Load game_dates.csv
df = pd.read_csv("game_dates.csv")

# Convert season and filter from 2017 onward
df['season_end_year'] = df['season'].str.split('-').str[0].astype(int) + 1
filtered_df = df[df['season_end_year'] >= 2024]

# Unique date/season pairs
unique_dates = filtered_df[['date', 'season']].drop_duplicates()
unique_dates=unique_dates.head(1)
# Change mode here: 'Offense' or 'Defense'
mode = 'Offense'  # Change to 'Defense' if needed

all_data = []

for _, row in unique_dates.iterrows():
    date = row['date']
    season = row['season']
    print(f"Pulling {mode} matchups for {date} in season {season}")
    
    try:
        matchup_df = get_omatchups_by_date(season, date, mode=mode)
        matchup_df['game_date'] = date
        matchup_df['season'] = season
        matchup_df['mode'] = mode
        all_data.append(matchup_df)
    except Exception as e:
        print(f"Failed to pull data for {date} in {season}: {e}")
    
    time.sleep(0.7)  # Be kind to the NBA API

# Combine
final_df = pd.concat(all_data, ignore_index=True)

# Save
filename = f"matchups_by_date_{mode.lower()}.csv"
final_df.to_csv(filename, index=False)
print(f"Done. Saved to {filename}")


Pulling Offense matchups for 20231024 in season 2023-24
Done. Saved to matchups_by_date_offense.csv
